In [1]:
import feedparser
import schedule
import time
import spacy
import pymongo
import time

In [2]:
nlp = nlp = spacy.load('pt')

In [3]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
rss = client["rss"]
news = rss["news"]

In [4]:
def get_ents(ents):
    result_ents = []
    for ent in ents:
        result_ents.append(str(ent))
    return result_ents

In [5]:
def get_obj(entries):
    list_object_to_insert = []
    for entry in entries:
        #print(nlp(entry.title).ents)
        if (news.find_one({"_id":str(entry.id)})) is None:
            list_object_to_insert.append({"_id": str(entry.id),
                         "extracted": get_ents(nlp(entry.title).ents),
                        "timestamp":time.time()})  
    return list_object_to_insert

In [6]:
def get_news():
    NewsFeed = feedparser.parse("http://g1.globo.com/dynamo/rss2.xml")
    entries = NewsFeed.entries
    save_objects = get_obj(entries)
    for obj in save_objects:
        news.insert_one(obj)
    print("Connected, found {} more news".format(len(save_objects)))

In [7]:
schedule.every(1).minutes.do(get_news)
while 1:
    schedule.run_pending()
    time.sleep(1)


Connected, found 0 more news
Connected, found 0 more news
Connected, found 0 more news
Connected, found 0 more news
Connected, found 0 more news
Connected, found 1 more news
Connected, found 0 more news
Connected, found 0 more news


KeyboardInterrupt: 